In [1]:
from io import StringIO
from bs4 import BeautifulSoup
from tika import parser
import re

file_path = "../Books Index for Phrases/CFA2019-L1V1.pdf"

file_data = []
_buffer = StringIO()
data = parser.from_file(file_path, xmlContent=True)
xhtml_data = BeautifulSoup(data['content'])
for page, content in enumerate(xhtml_data.find_all('div', attrs={'class': 'page'})):
    print('Parsing page {} of pdf file...'.format(page+1))
    _buffer = StringIO()
    _buffer.write(str(content))
    parsed_content = parser.from_buffer(_buffer.getvalue())
    _buffer.truncate()
    file_data.append({'id': 'page_'+str(page+1), 'content': parsed_content['content']})

Parsing page 1 of pdf file...
Parsing page 2 of pdf file...
Parsing page 3 of pdf file...
Parsing page 4 of pdf file...
Parsing page 5 of pdf file...
Parsing page 6 of pdf file...
Parsing page 7 of pdf file...
Parsing page 8 of pdf file...
Parsing page 9 of pdf file...
Parsing page 10 of pdf file...
Parsing page 11 of pdf file...
Parsing page 12 of pdf file...
Parsing page 13 of pdf file...
Parsing page 14 of pdf file...
Parsing page 15 of pdf file...
Parsing page 16 of pdf file...
Parsing page 17 of pdf file...
Parsing page 18 of pdf file...
Parsing page 19 of pdf file...
Parsing page 20 of pdf file...
Parsing page 21 of pdf file...
Parsing page 22 of pdf file...
Parsing page 23 of pdf file...
Parsing page 24 of pdf file...
Parsing page 25 of pdf file...
Parsing page 26 of pdf file...
Parsing page 27 of pdf file...
Parsing page 28 of pdf file...
Parsing page 29 of pdf file...
Parsing page 30 of pdf file...
Parsing page 31 of pdf file...
Parsing page 32 of pdf file...
Parsing page 33 o

## Glossary

In [2]:
## Glossary ## 
## Page 763 (G-1) to Page 797 (G-35)
glossary_pages = xhtml_data.find_all('div', attrs={'class': 'page'})[762: ]


In [3]:
glossary_page_3 = glossary_pages[3]

In [4]:
glossary_page_3.text

'\nG-4 Glossary\n\nBond equivalent yield  A calculation of yield that is annualized \nusing the ratio of 365 to the number of days to maturity. \nBond equivalent yield allows for the restatement and com-\nparison of securities with different compounding periods.\n\nBond indenture  The governing legal credit agreement, typ-\nically incorporated by reference in the prospectus. Also \ncalled trust deed.\n\nBond market vigilantes  Bond market participants who might \nreduce their demand for long- term bonds, thus pushing \nup their yields.\n\nBond yield plus risk premium approach  An estimate of the \ncost of common equity that is produced by summing the \nbefore- tax cost of debt and a risk premium that captures the \nadditional yield on a company’s stock relative to its bonds. \nThe additional yield is often estimated using historical \nspreads between bond yields and stock yields.\n\nBonus issue of shares  A type of dividend in which a com-\npany distributes additional shares of its com

In [5]:
key_phrase = []
explanation = []

text = glossary_page_3.text.split('\n')
for line in text:
    
    if not line:
        continue

    line = line.replace('’', '')

    ## remove page number
    if re.search(r'G-\d{1,2}', line): 
        continue

    ## remove Glossary 
    if re.search(r'Glossary', line): 
        continue

    ## © CFA Institute. For candidate use only. Not for distribution.
    if re.search(r'CFA Institute. For candidate use only', line): 
        continue
    
    ## check if there is key word in line
    if '  ' in line:
        current_key_phrase, current_explanation = line.split('  ')
        key_phrase.append(current_key_phrase)
        explanation.append(current_explanation)
    else:
        # print(explanation)
        explanation[-1] = explanation[-1] + line

    # text = text.replace('\n', '').strip()
    # text = text.split('  ')
    
    # if text != ['']:
    #     print(text)


In [6]:
TEXT = []
PHRASES = []
EXPLANATION = []

for page in glossary_pages:

    text = page.text.split('\n')

    for line in text:
        # print(p.text)
        if not line:
            continue
        
        line = line.replace('’', "'")
        line = line.replace('–', '-')
        line = line.replace('—', '-')
        line = line.replace('- ', '-')
        line = line.replace('“', '"')
        line = line.replace('”', '"')

        ## remove page number
        if re.search(r'G-\d{1,2}', line): 
            continue

        ## remove Glossary 
        if re.search(r'Glossary', line): 
            continue

        ## © CFA Institute. For candidate use only. Not for distribution.
        if re.search(r'CFA Institute. For candidate use only', line): 
            continue
        
        ## check if there is key word in line
        if '  ' in line:
            current_key_phrase, current_explanation = line.split('  ')
            PHRASES.append(current_key_phrase)
            EXPLANATION.append(current_explanation)
        else:
            # print(explanation)
            EXPLANATION[-1] = EXPLANATION[-1] + line
        



In [7]:
for phrase, explanation in zip(PHRASES, EXPLANATION):
    original_extraction = phrase + '  ' + explanation
    TEXT.append(original_extraction)

In [8]:
import pandas as pd 
df = pd.DataFrame({'ID': [i for i in range(len(PHRASES))],
            'original_glossary_extracted': TEXT,
            'key_phrase': PHRASES,
            'source_book': ['CFA2019-L1V1' for _ in range(len(PHRASES))],
            'definition_or_explanation': EXPLANATION})

In [9]:
# df.to_csv('glossary_extraction_2013_Bodie.Kane.Marcus_Investments.csv', index=False)

In [10]:
df.head()

,ID,original_glossary_extracted,key_phrase,source_book,definition_or_explanation
0,0,A priori probability A probability based on l...,A priori probability,CFA2019-L1V1,A probability based on logical analysis rather...
1,1,Abnormal return The amount by which a securit...,Abnormal return,CFA2019-L1V1,The amount by which a security's actual return...
2,2,Absolute advantage A country's ability to pro...,Absolute advantage,CFA2019-L1V1,A country's ability to produce a good or servi...
3,3,Absolute dispersion The amount of variability...,Absolute dispersion,CFA2019-L1V1,The amount of variability present without comp...
4,4,Absolute frequency The number of observations...,Absolute frequency,CFA2019-L1V1,The number of observations in a given interval...


## Reformat extraction

In [11]:
import re
def find_full_expression_and_abbreviation(text):
    text_length = len(text)
    match = re.search('\(.+\)', text) # search of bracket
    match_1 = re.search('[A-Z]{2,8}', text) # search for Capitalization
    match_2 = re.search('[A-Z]/[A-Z]', text) # search for Capitalization
    if match and (match_1 or match_2):
        content_in_bracket_length = len(match.group(0)) - 2
    else:

        return [text]

    start, end = re.search('\(.+\)', text).span()
    if content_in_bracket_length > text_length / 2:
        
        full_expression = text[start+1: end-1]
        abbreviated_expression = text[: start-1]
    else:
        full_expression = text[: start-1]
        abbreviated_expression = text[start+1: end-1]

    return [full_expression, abbreviated_expression]

In [12]:
text = 'Automated Clearing House (ACH)'
find_full_expression_and_abbreviation(text)


['Automated Clearing House', 'ACH']

In [13]:
full = []
extracted_acronym_1 = []

for phrase, definition in zip(df['key_phrase'].values, df['definition_or_explanation'].values):
     res = find_full_expression_and_abbreviation(phrase)
     if len(res) == 1:
         full.append(res[0])
         extracted_acronym_1.append('')
         # extracted_acronym_2.append('')

     if len(res) == 2:
         full.append(res[0])
         extracted_acronym_1.append(res[1])
         # extracted_acronym_2.append('')

In [14]:
import re
def find_abbreviation_in_explanation(text):
    text = text.strip()
    text_length = len(text)
    match = re.search('^\([A-Z]{2,8}\)', text) # search of bracket
    if match:
        content_in_bracket_length = len(match.group(0)) - 2
    else:
        return [text]

    start, end = re.search('^\([A-Z]{2,8}\)', text).span()

    full_expression = text[end+1: ]
    abbreviated_expression = text[start+1: end-1]

    return [full_expression, abbreviated_expression]

In [15]:
text = '(CAL) A graph line that describes the combinations of expected return and standard deviation of return available to an investor from combining the optimal portfolio of risky assets with the risk-free asset.'

In [16]:
re.search('^\([A-Z]{2,8}\)', text)

<re.Match object; span=(0, 5), match='(CAL)'>

In [17]:
find_abbreviation_in_explanation(text)

['A graph line that describes the combinations of expected return and standard deviation of return available to an investor from combining the optimal portfolio of risky assets with the risk-free asset.',
 'CAL']

In [18]:
full_explanation = []
extracted_acronym_2 = []

for phrase, definition  in zip(df['key_phrase'].values, df['definition_or_explanation'].values):
     res = find_abbreviation_in_explanation(definition)
     if len(res) == 1:
         full_explanation.append(res[0])
         extracted_acronym_2.append('')

     if len(res) == 2:
         full_explanation.append(res[0])
         extracted_acronym_2.append(res[1])

In [19]:
for i in range(len(extracted_acronym_2)):
    if extracted_acronym_2[i] != '':
        extracted_acronym_1[i] = extracted_acronym_2[i]

In [20]:
df['key_phrase'] = full
df['definition_or_explanation'] = full_explanation
df['extracted_acronym_1'] = extracted_acronym_1
df['extracted_acronym_2'] = ['' for _ in range(len(extracted_acronym_2))]

In [21]:
def clean_extracted_key_phrase(phrase):
    try: 
        phrase = phrase.replace('-', ' ')
        phrase = phrase.replace('/', ' ')
        return phrase

    except:
        return phrase

In [22]:
df['key_phrase'] = df['key_phrase'].apply(clean_extracted_key_phrase)

In [23]:
df.columns

Index(['ID', 'original_glossary_extracted', 'key_phrase', 'source_book',
       'definition_or_explanation', 'extracted_acronym_1',
       'extracted_acronym_2'],
      dtype='object')

In [24]:
df = df[['ID', 'original_glossary_extracted', 'key_phrase', 'extracted_acronym_1',
       'extracted_acronym_2', 'definition_or_explanation', 'source_book',
        ]]

In [25]:
df.to_csv('glossary_extraction_CFA2019-L1V1.csv', index=False)